TODO - DESCRIBE THIS FILE 

In [ ]:
import json
import math
import os
import shutil
import csv

from PIL import Image, ImageDraw
from PIL.TiffTags import TAGS
from tqdm import tqdm

from utils.utils import make_patches

# Dataset Paths (Remember to replace placeholders with your actual paths)
DATASET_SOURCE = os.path.join(
    "/mnt", "f", "dataUtils", "raw_data"
)  # FIXME: Change accordingly
OUTPUT_DATASET = os.path.join("/mnt", "e", "dataset")  # FIXME: Change accordingly

In [ ]:
def convert_cxcywh_to_xyxy(box):
    """Converts a bounding box from center-x, center-y, width, height format (CXCYWH)
    to top-left-x, top-left-y, bottom-right-x, bottom-right-y format (XYXY).

    Args:
        box: A tuple representing the bounding box in CXCYWH format.

    Returns:
        tuple: A tuple representing the bounding box in XYXY format.
    """
    cx, cy, w, h = box
    xmin = cx - w // 2
    ymin = cy - h // 2
    xmax = cx + (w + 1) // 2  # ceil
    ymax = cy + (h + 1) // 2  # ceil
    return xmin, ymin, xmax, ymax


def get_image_metadata(img):
    """Extracts resolution metadata from a TIFF image.

    Args:
        img: A PIL Image object.

    Returns:
        dict[str, float]: A dictionary containing resolution information (x, y, and unit).
    """
    tiff_tags = {TAGS.get(tag, tag): value for tag, value in img.tag.items()}

    res = {
        "x_resolution": tiff_tags["XResolution"][0][0] / tiff_tags["XResolution"][0][1],
        "y_resolution": tiff_tags["YResolution"][0][0] / tiff_tags["YResolution"][0][1],
        "resolution_unit": tiff_tags["ResolutionUnit"],
    }
    return res


class ImageUtils:
    """Provides utility functions for working with image resolutions and areas."""

    def __init__(self, img):
        self.img = img
        self.metadata = get_image_metadata(img)
        self.x_resolution = float(self.metadata["x_resolution"])
        self.y_resolution = float(self.metadata["y_resolution"])

    def area_units_to_pixels(self, area):
        """Converts area from resolution units to pixels."""
        return float(area) * (max(self.x_resolution, self.y_resolution) ** 2)

    def area_pixels_to_units(self, area):
        """Converts area from pixels to resolution units."""
        return float(area) / (max(self.x_resolution, self.y_resolution) ** 2)

In [ ]:
def sanitize_annotations(name, annotation) -> None:
    """Sanitizes annotations by removing duplicates and invalid bounding boxes.

    Args:
        name: The name of the image associated with the annotations.
        annotations: A dictionary containing "category_id" and "boxes" lists.

    Returns:
        A dictionary containing sanitized "category_id" and "boxes" lists.
    """
    sanitized_annotations = {"category_id": [], "boxes": []}

    bbox_dict = dict()
    for label, bbox in zip(annotation["category_id"], annotation["boxes"]):
        if bbox in bbox_dict:
            if bbox_dict[bbox] == label:
                print(f"WARNING: Duplicate bbox found in {name}: {bbox}")
            else:
                print(f"ERROR: Same bbox with different label found in {name}: {bbox}")
            continue
        if any(coord < 0 for coord in bbox):
            print(f"Corrupted box found in {name}: {bbox}")
            continue

        bbox_dict[bbox] = label

    for key in bbox_dict:
        sanitized_annotations["category_id"].append(bbox_dict[key])
        sanitized_annotations["boxes"].append(key)

    return sanitized_annotations


def convert_annotations(csv_path, img_path):
    """Converts a CSV file and an image into annotations.
    Args:
        csv_path: The path to the CSV file containing annotations.
        img_path: The path to the image file.
    Returns:
        A dictionary containing "category_id" and "boxes" lists representing the annotations.
    """
    annotations = {"category_id": [], "boxes": []}

    util = ImageUtils(Image.open(img_path))
    if os.path.exists(csv_path):
        with open(csv_path) as data_file:
            data = csv.reader(data_file)
            next(data)  # Skip header row
            for row in data:
                try:
                    _, label, area, category_id, _ = (
                        row  # Unpack row, ignore filename and category_name
                    )
                    _, coordinates = label.split(":")  # Extract coordinates from label
                    y, x = coordinates.split("-")
                    y, x = int(y), int(x)

                    area_in_pixels = util.area_units_to_pixels(float(area))
                    bbox_side = int(
                        math.sqrt(area_in_pixels / math.pi) * 2
                    )  # Calculate square side length

                    bbox = convert_cxcywh_to_xyxy((x, y, bbox_side, bbox_side))
                    annotations["category_id"].append(int(category_id))
                    annotations["boxes"].append(bbox)
                except Exception as e:
                    annotations["boxes"] = []
                    annotations["category_id"] = []

                    print(f"Error processing row in {csv_path}: {e}")
                    print(f"{csv_path} Failed")
                    break

    return annotations


def convert_dataset(data_path, imgs_path, res_path, replace_imgs=True):
    """Converts a dataset from the source format to the desired output format.
    Args:
        data_path: The path to the directory containing CSV annotation files
        imgs_path: The path to the directory containing image files
        res_path: The path to the output directory where the converted dataset will be saved
        replace_imgs: Whether to replace existing images in the output directory
    """
    img_list = [
        os.path.splitext(f)[0]
        for f in os.listdir(imgs_path)
        if os.path.isfile(os.path.join(imgs_path, f)) and f.endswith(".tif")
    ]

    if replace_imgs:
        try:
            shutil.rmtree(res_path)
        except FileNotFoundError:
            pass

    os.makedirs(os.path.join(res_path, "imgs"), exist_ok=True)

    dataset = {}
    image_id = 1  # Start image IDs from 1

    # Prepare to write stats to CSV
    stats_data = []

    for name in img_list:
        csv_path = f"{data_path}/{name}.tif.csv"
        img_path = f"{imgs_path}/{name}.tif"
        annotations = convert_annotations(csv_path, img_path)
        if not annotations["boxes"]:
            continue  # Skip images without annotations

        if replace_imgs:
            img = Image.open(img_path)
            img.save(os.path.join(res_path, "imgs", f"{image_id}.jpeg"), quality=100)

        annotations = sanitize_annotations(name, annotations)
        dataset[str(image_id)] = annotations

        # Calculate and store stats
        healthy_area = 0
        necrosed_area = 0
        healthy_count = 0
        necrosed_count = 0
        for bbox, class_id in zip(annotations["boxes"], annotations["category_id"]):
            if class_id == 1:  # Healthy area
                healthy_area += math.pi * ((bbox[2] - bbox[0]) / 2) ** 2
                healthy_count += 1
            else:  # Necrosed area
                necrosed_area += math.pi * ((bbox[2] - bbox[0]) / 2) ** 2
                necrosed_count += 1

        stats_data.append(
            [
                f"{image_id}.jpeg",
                int(healthy_area),
                int(necrosed_area),
                healthy_count,
                necrosed_count,
            ]
        )

        image_id += 1

    with open(os.path.join(res_path, "dataset.json"), "w") as outfile:
        json.dump(dataset, outfile, indent=1)

    # Write stats to CSV
    with open(os.path.join(res_path, "stats.csv"), "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(
            [
                "Image",
                "healthy area",
                "necrosed area",
                "healthy count",
                "necrosed count",
            ]
        )  # Write header
        writer.writerows(stats_data)

    print("Finished converting")


convert_dataset(
    f"{DATASET_SOURCE}/csv",
    f"{DATASET_SOURCE}/imgs",
    OUTPUT_DATASET,
    replace_imgs=False,
)

In [ ]:
def box_area(box):
    """Calculates the box area

    Args:
        box: The box in XYXY format.

    Returns:
        float: The area of the box.
    """
    x1, y1, x2, y2 = box
    return max(0, x2 - x1) * max(0, y2 - y1)


def patch_annots(cropped_img, crop_box, annots, crop_tolerance, erase_cropped=True):
    """Crops an image and adjusts annotations based on the crop region.

    Args:
        cropped_img (Image.Image): The PIL Image object to crop
        crop_box (Box): The bounding box defining the crop region in XYXY format
        annots (dict[str, Any]): A dictionary containing "category_id" and "boxes"
                                 lists representing annotations
        crop_tolerance (float): The tolerance for considering an annotation as fully
                                within the crop
        erase_cropped (bool, optional): Whether to draw over partially cropped annotations
                                        on the image. Defaults to True.

    Returns:
        tuple[Image.Image, tuple[list[int], list[Box]]]: A tuple containing the cropped
                                                        image and the adjusted annotations
    """

    cropped_annots = {"category_id": [], "boxes": []}
    labels, bboxes = annots["category_id"], annots["boxes"]

    draw_context = ImageDraw.Draw(cropped_img)
    for label, bbox in zip(labels, bboxes):
        relative_bbox = (
            max(crop_box[0], bbox[0]) - crop_box[0],  # xmin
            max(crop_box[1], bbox[1]) - crop_box[1],  # ymin
            min(crop_box[2], bbox[2]) - crop_box[0],  # xmax
            min(crop_box[3], bbox[3]) - crop_box[1],
        )  # ymax
        cropped = 1 - box_area(relative_bbox) / box_area(bbox)
        if cropped <= crop_tolerance:
            cropped_annots["category_id"].append(label)
            cropped_annots["boxes"].append(relative_bbox)
        elif cropped < 1 and erase_cropped:
            draw_context.rectangle(relative_bbox, width=1, fill="black")

    return cropped_annots


def patch_sample(img, annots, desired_image_size, overlap, crop_tolerance):
    """Patches a large image into smaller images with adjusted annotations.

    Args:
        img (Image.Image): The PIL Image object to patch
        annots (dict[str, Any]): A dictionary containing "category_id" and "boxes"
                                 lists representing annotations
        desired_image_size (int): The desired size of each patch
        overlap (float): The overlap between adjacent patches (as a fraction of
                         `desired_image_size`)
        crop_tolerance (float): The tolerance for considering an annotation as fully
                                within a patch

    Returns:
        tuple[list[Image.Image], list[tuple[list[int], list[Box]]]]: A tuple containing
                                                                    a list of patched
                                                                    images and a list of
                                                                    corresponding
                                                                    adjusted annotations
    """

    padded_width, padded_height, patch_boxes = make_patches(
        img.width, img.height, desired_image_size, overlap
    )
    padded_img = Image.new("RGB", (padded_width, padded_height))
    padded_img.paste(img)
    res_imgs, res_annots = [], []

    for patch_box in patch_boxes:
        patched_img = padded_img.crop(patch_box)
        patched_annots_ = patch_annots(patched_img, patch_box, annots, crop_tolerance)
        res_imgs.append(patched_img)
        res_annots.append(patched_annots_)

    return res_imgs, res_annots


def patch_dataset(
    dataset_root, desired_image_size=1024, overlap=0.2, crop_tolerance=0.3
):
    """Patches images in a dataset and saves the patched images and annotations.

    Args:
        dataset_root (str): The root directory of the dataset
        desired_image_size (int, optional): The desired size of each patch. Defaults to 1024.
        overlap (float, optional): The overlap between adjacent patches (as a fraction of
                                    `desired_image_size`). Defaults to 0.2.
        crop_tolerance (float, optional): The tolerance for considering an annotation as
                                        fully within a patch. Defaults to 0.3.
    """
    dataset_root = os.path.normpath(dataset_root)
    annot_file = os.path.join(dataset_root, "dataset.json")
    imgs_dir = os.path.join(dataset_root, "imgs")
    dataset_parent = os.path.dirname(dataset_root)
    patched_root = os.path.join(
        dataset_parent,
        os.path.basename(dataset_root)
        + "_patched_croptolerance="
        + str(crop_tolerance),
    )
    os.makedirs(os.path.join(patched_root, "imgs"), exist_ok=True)

    patched_dataset = {}

    with open(annot_file, "r") as annot_file:
        dataset = json.load(annot_file)

    for image_id, annotations in tqdm(dataset.items()):

        img = Image.open(os.path.join(imgs_dir, f"{image_id}.jpeg"))
        patched_imgs, patched_annots = patch_sample(
            img, annotations, desired_image_size, overlap, crop_tolerance
        )
        patch_num = 1

        for patched_image, patched_annotations in zip(patched_imgs, patched_annots):
            patch_id = f"{image_id}_{patch_num}"
            patched_dataset[patch_id] = patched_annotations
            patched_image.save(
                os.path.join(patched_root, "imgs", patch_id + ".jpeg"), quality=95
            )
            patch_num += 1

    with open(os.path.join(patched_root, "dataset.json"), "w") as outfile:
        json.dump(patched_dataset, outfile, indent=1)


patch_dataset(OUTPUT_DATASET, crop_tolerance=0.7)